This function will read the files with Mathemaica Pythonized expressions and save them to the arrays of strings

In [13]:
import numpy as np

In [14]:
def import_expressions(file):
    file_expr=open(file,"r")
    list_expr=[line.rstrip() for line in file_expr]
    file_expr.close()
    return list_expr

In [26]:
files_EOMs=("EOMsI_py.txt","EOMsT_py.txt","EOMsB_py.txt")
files_Coefs=("CoefsI_py.txt","CoefsT_py.txt","CoefsB_py.txt")

In [33]:
class Mathematica_importer:
    def __init__(self, files_EOMs, files_Coefs):
        self._func_names={'sqrt':np.sqrt, 'cosh':np.cosh, 'pi':np.pi, 'sinh':np.sinh, 'tanh':np.tanh, 'exp':np.exp, 'cos':np.cos, 'sin':np.sin} 
        self._sets_of_eoms=[self.import_expressions(file) for file in files_EOMs]
        self._sets_of_coefs=[self.import_expressions(file) for file in files_Coefs]
    
    def import_expressions(self,file):
        file_expr=open(file,"r")
        list_expr=[line.rstrip() for line in file_expr]
        file_expr.close()
        return list_expr
    
    def exprs_eoms(self):
        return self._sets_of_eoms
    
    def exprs_coefs(self):
        return self._sets_of_coefs
    
    def renaming_rules(self):
        return self._func_names

In [34]:
math_import=Mathematica_importer(files_EOMs, files_Coefs)

In [36]:
class PDE_2d_problem:
 
    def __init__(self,  importer, grid_vars, phys_vars):
        # This sets the names of functions used in Mathematica to the known functions in NumPy
        self._func_names=importer.renaming_rules()# Load the Pythonized Mathematica expressions for Internal, Top, Bottom equations and Internal, Top, Bottom coefficients of Jacobian
        self._eomsI_expr, self._eomsT_expr,  self._eomsB_expr = importer.exprs_eoms()
        self._coefsI_expr, self._coefsT_expr, self._coefsB_expr= importer.exprs_coefs()
        
        self.ne=len(self._eomsI_expr)
        
        # Set the internal values to the provided grid size and save physical parameters
        self.nx, self.ny=grid_vars
        self.phys_vars=phys_vars
        
        #Create grid via built in function
        self.gy, self.gx =self.create_grid()
        
        #Slice the grid for Internal, Top, Bottom calculations
        self.sliced_grid=self.slice_grid()
          
    def create_grid(self):
        gy = np.reshape(np.random.rand(self.ny),(1,self.ny))
        gx = np.reshape(np.random.rand(self.nx),(self.nx,1))
        return(gy,gx)
        
    
    def slice_gfm(self,gfm):
        ndif=7
        gft=np.reshape(gfm[:,:,:,-1],(self.ne,ndif,self.nx,1))
        gfb=np.reshape(gfm[:,:,:,0],(self.ne,ndif,self.nx,1))
        gfi=gfm[:,:,:,1:-1]
        return(gft,gfi,gfb)
    
    def slice_grid(self):
        gy=self.gy
        gx=self.gx
        gxi=gx
        gxt=gx
        gxb=gx
        gyi=gy[:,1:-1]
        gyt=gy[-1]
        gyb=gy[0]
        return({'gxb':gxb,'gxi':gxi,'gxt':gxt,'gyb':gyb,'gyi':gyi,'gyt':gyt})
    
    def EOMs(self,gfm):
        # Slice the input for Internal, Top, Bottom calculations
        (gft,gfi,gfb)=self.slice_gfm(gfm)
        # Include all internal definitions into the local dictionary
        locals().update(self.phys_vars)
        locals().update(self.sliced_grid)
        locals().update(self._func_names)
        
        # Evaluate the Internal, Top, Bottom expressions using internal definitions
        eomsT=np.zeros((self.ne,self.nx,1),dtype=float)
        for i, expr in enumerate(self._eomsT_expr):           
            eomsT[i]=eval(expr,globals(),locals())
        
        eomsI=np.zeros((self.ne,self.nx,self.ny-2),dtype=float)
        for i, expr in enumerate(self._eomsI_expr):           
            eomsI[i]=eval(expr,globals(),locals())
            
        eomsB=np.zeros((self.ne,self.nx,1),dtype=float)
        for i, expr in enumerate(self._eomsB_expr):           
            eomsB[i]=eval(expr,globals(),locals())
        
        # Create the full matrix of equations
        eomsM=np.concatenate((eomsB,eomsI,eomsT),axis=2)
        
        return(eomsM)
    

In [37]:
problem=PDE_2d_problem(math_import,(100,100), some_ph)

TypeError: __init__() missing 1 required positional argument: 'phys_vars'

In [39]:
%%timeit
some_phys_vars={'c1':9.9, 'p0':1.2, 'mu1':1., 'mu':2, 'a0':0.3, 'nperiods':1}
some_lattice=(10,10)
some_gfm=np.random.rand(9,7,some_lattice[0],some_lattice[1])
problem=PDE_2d_problem(math_import,some_lattice,some_phys_vars)
problem.EOMs(some_gfm);

10 loops, best of 3: 50 ms per loop


In [ ]:
import

In [40]:
b

6